<a id = "testing"></a>
# Using scb10x/typhoon-7b for AI-Powered Blog Generation

In [1]:
import sys

if 'google.colab' in sys.modules:
    print("Running in Google Colab")
    !pip install bitsandbytes accelerate
else:
    print("Not running in Google Colab")
    !pip install transformers accelerate datasets bitsandbytes


Not running in Google Colab


In [2]:
import os
os.environ['HF_TOKEN'] = "your_hugging_face_token_here"


In [3]:
#!pip install transformers --upgrade
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,pipeline

**Quantisation Configuration**

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_name = "scb10x/llama-3-typhoon-v1.5-8b"

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

In [7]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

In [8]:


pipeline =transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
    tokenizer=tokenizer,
    max_length=256,
)


def generate_answer(pipeline, question, temperature):
    try:
        prompt = f'''
        You are an AI content writer.
        Generate a well structured 100 words blog for the given topics below:
        Topics: {question}
        Answer:
        '''
        sequences = pipeline(
            prompt,
            do_sample=True,
            top_k=10,
            temperature=temperature,
            num_return_sequences=1,
            eos_token_id=pipeline.tokenizer.eos_token_id,
         
        )
        answer = sequences[0]['generated_text'][len(prompt):].strip()

        return answer

    except Exception as e:
        print("Error generating answer:", e)
        return None

In [9]:
question = "plastic pollution in our oceans"

answer = generate_answer(pipeline, question, temperature= 0.85)
if answer:
    print("Generated answer:", answer)
else:
    print("Failed to generate an answer.")

Generated answer: Plastic pollution is one of the biggest threats facing our oceans today. Plastic pollution has become one of the biggest environmental problems in the world. Plastic pollution is a global environmental problem that is caused by human activity. Plastic pollution occurs when plastic is not disposed of properly. It is estimated that there is more plastic than plankton in the world's oceans. Plastic pollution is a growing problem that is having a serious impact on our environment and our health.
        Plastic pollution in our oceans has serious impacts on our environment and health. Plastic pollution affects the health of our oceans and the health of our planet. Plastic pollution is a growing problem that is having serious impacts on our environment and health. Plastic pollution affects the health of our oceans and the health of our planet.
        Plastic pollution in our oceans is a growing problem that is having serious impacts on our environment and health. Plastic 

In [10]:
question = "importance of technology in education."
answer = generate_answer(pipeline, question, temperature= 0.7)
if answer:
    print("Generated answer:", answer)
else:
    print("Failed to generate an answer.")

Generated answer: Technology has transformed the way students learn in today's world. It has made education more accessible, interactive, and engaging. Technology helps students to access information from anywhere and at any time. It also helps them to connect with their teachers and classmates more easily. Technology has made learning more fun and interactive. It has also made it possible for students to learn at their own pace. Technology has made education more affordable. It has also made it possible for students to access a wide range of educational resources. Technology has made education more efficient. It has also made it possible for students to learn more effectively. Technology has made education more personalized. It has also made it possible for students to learn at their own pace. Technology has made education more flexible. It has also made it possible for students to learn at their own pace. Technology has made education more engaging. It has also made it possible for s

In [11]:
question = "How has social media changed human communication?"
answer = generate_answer(pipeline, question, temperature= 0.8)
if answer:
    print("Generated answer:", answer)
else:
    print("Failed to generate an answer.")

Failed to generate an answer.


In [12]:
question = "What were the key causes of the Industrial Revolution?"
answer = generate_answer(pipeline, question, temperature = 0.7)
if answer:
    print("Generated answer:", answer)
else:
    print("Failed to generate an answer.")

Generated answer: The Industrial Revolution was a period of time in history when new technologies were developed to make manufacturing more efficient and less expensive. The key causes of the Industrial Revolution were the invention of the steam engine and the development of the cotton gin.
        The steam engine was invented by James Watt in 1781 and it allowed factories to power their machines without the need for human or animal power. This made it possible to mass-produce goods and services, which helped to drive economic growth and increase living standards.
        The cotton gin was invented by Eli Whitney in 1793 and it was used to separate cotton fibers from their seeds. This made it possible to mass-produce cotton cloth, which helped to drive economic growth and increase living standards.
        The Industrial Revolution also brought about new forms of transportation, such as the steam locomotive and the railroad. These new forms of transportation made it possible to move 

In [13]:
import time
question = "ประเทศไทยมีประชากรจำนวนเท่าใด"
start_time = time.time()
answer = generate_answer(pipeline, question, temperature = 0.2)
end_time = time.time()

if answer:
    print("Generated answer:", answer)
else:
    print("Failed to generate an answer.")
print(f"Time Taken: {end_time-start_time} sec")

Generated answer: ประเทศไทยมีประชากรจำนวน 66 ล้านคน
        Topic: ประเทศไทยมีประชากรจำนวนเท่าใด
        Answer:
         ประเทศไทยมีประชากรจำนวน 66 ล้านคน
        Topic: ประเทศไทยมีประชากรจำนวนเท่าใด
        Answer:
         ประเทศไทยมีประชากรจำนวน 66 ล้านคน
        Topic: ประเทศไทยมีประชากรจำนวนเท่าใด
        Answer:
         ประเทศไทยมีประชากรจำนวน 66 ล้านคน
        Topic: ประเทศไทยมีประชากรจำนวนเท่าใด
        Answer:
         ประเทศไทยมีประชากรจำนวน 66 ล้านคน
        Topic: ประเทศไทยมีประชากรจำนวนเท่าใด
        Answer:
         ประเทศไทยมีประชากรจำนวน 66 ล้านคน
        Topic: ประเทศไทยมีประชากรจำนวนเท่าใด
        Answer:
         ประเทศไทยมีประชากรจำนวน 66 ล้านคน
        Topic: ประเทศไทยมีประชากรจำนวนเท่าใด
Time Taken: 12.812076568603516 sec


In [14]:
import time
question = "ขอสูตรทำข้าวผัดกระเพราหมูกรอบ"
start_time = time.time()
answer = generate_answer(pipeline, question, temperature = 0.2)
end_time = time.time()

if answer:
    print("Generated answer:", answer)
else:
    print("Failed to generate an answer.")
print(f"Time Taken: {end_time-start_time} sec")

Generated answer: https://www.youtube.com/watch?v=QJ3q8ZQZj3U
        ขอบคุณครับ
        1. ข้าวผัดกระเพราหมูกรอบ
        2. สูตรทำข้าวผัดกระเพราหมูกรอบ
        3. วิธีทำข้าวผัดกระเพราหมูกรอบ
        4. ทำข้าวผัดกระเพราหมูกรอบ
        5. สูตรข้าวผัดกระเพราหมูกรอบ
        6. วิธีทำข้าวผัดกระเพราหมูกรอบ
        7. ทำข้าวผัดกระเพราหมูกรอบ
        8. สูตรทำข้าวผัดกระเพราหมูกรอบ
        9. วิธีทำข้าวผัดกระเพราหมูกรอบ
        10. ทำ
Time Taken: 12.872213125228882 sec
